In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, auc, classification_report



In [2]:
df = pd.read_csv('processed_data/merged_df.csv')

In [8]:
# split

target_column = 'netcontractsigned'
X = df.drop(target_column, axis=1)
y = df[target_column]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



#### First Strategy

Class Weights

In [11]:

# Calculate AUC-PR (more reliable for imbalanced data)
#precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
auc_pr = auc(recall, precision)

# Test class weights with your strong signal
print("\n=== TESTING CLASS WEIGHTS ===")

# Calculate balanced class weight
pos_weight = (1 - y_train.mean()) / y_train.mean()  # About 42 for your data
print(f"Calculated positive class weight: {pos_weight:.1f}")

# Train with class weights
lr_weighted = LogisticRegression(
    class_weight={0: 1, 1: pos_weight},
    random_state=42, 
    max_iter=1000
)
lr_weighted.fit(X_train_scaled, y_train)

# Get predictions
y_pred_weighted = lr_weighted.predict_proba(X_test_scaled)[:, 1]

# Evaluate
precision_w, recall_w, _ = precision_recall_curve(y_test, y_pred_weighted)
auc_pr_weighted = auc(recall_w, precision_w)
auc_roc_weighted = roc_auc_score(y_test, y_pred_weighted)

print(f"\nClass Weights Results:")
print(f"AUC-ROC: {auc_roc_weighted:.3f}")
print(f"AUC-PR: {auc_pr_weighted:.3f}")
print(f"Improvement over baseline: {auc_pr_weighted/auc_pr:.2f}x")

# Check different thresholds
for thresh in [0.1, 0.2, 0.3]:
    pred_at_thresh = (y_pred_weighted >= thresh).astype(int)
    if pred_at_thresh.sum() > 0:
        precision_at_thresh = (y_test[pred_at_thresh == 1]).mean()
        print(f"At {thresh} threshold: {pred_at_thresh.sum()} predicted positive, {precision_at_thresh:.3f} precision")

NameError: name 'recall' is not defined